In [154]:
import torch
from torch import nn
from simulation.unity_simulator.comm_unity import UnityCommunication
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import matplotlib.pyplot as plt
import json
import math
import csv
import os
import glob

In [9]:
# from simulation.evolving_graph.execut import ScriptExecutor
from simulation.evolving_graph import check_programs
import dataset_utils.add_preconds as add_preconds
from simulation.evolving_graph import utils
from simulation.evolving_graph.scripts import (
    Action,
    ScriptParseException, Script, ScriptLine, ScriptObject,
    read_script_from_string, read_script_from_list_string,
)
import regex as re

In [10]:
from dataset_utils.execute_script_utils import (
    parse_exec_script_file, render_script,
    obtain_scene_id_from_path, render_script_from_path,
)
from unity_simulator.comm_unity import UnityCommunication
comm = UnityCommunication()

In [11]:
env_id = 0

In [12]:
comm.reset(env_id)

True

In [24]:
comm.add_character()

True

In [25]:
_, graph_input = comm.environment_graph()
graph_input = check_programs.translate_graph_dict_nofile(graph_input)

In [17]:
def read_script_file(filename):
    with open(filename) as f:
        title = f.readline().strip()
        assert '[' not in title
        description = f.readline().strip()
        assert '[' not in description
        script_lines = []
        script_lines_str = []
        index = 1
        for line in f:
            if '[' not in line:
                continue
            line = line.strip()
            if len(line) > 0 and not line.startswith('#'):
                params = []

                patt_action = r'^\[(\w+)\]'
                patt_params = r'\<(.+?)\>\s*\((.+?)\.(.+?)\)'

                action_match = re.search(patt_action, line.strip())
                if not action_match:
                    raise ScriptParseException('Cannot parse action')
                action_string = action_match.group(1).upper()
                if action_string not in Action.__members__:
                    raise ScriptParseException('Unknown action "{}"', action_string)
                action = Action[action_string]

                param_match = re.search(patt_params, action_match.string[action_match.end(1):])
                while param_match:
                    params.append(ScriptObject(param_match.group(1), int(param_match.group(2))))
                    param_match = re.search(patt_params, param_match.string[param_match.end(2):])

                if len(params) != action.value[1]:
                    raise ScriptParseException('Wrong number of parameters for "{}". Got {}, expected {}',
                                               action.name, len(params), action.value[1])
                script_lines.append(ScriptLine(action, params, index))
                script_lines_str.append('[{}]'.format(action.name) + ''.join([' ' + str(par) for par in params]))
                index += 1
    return title, description, Script(script_lines), script_lines_str

In [18]:
title, description, script, script_str = read_script_file(
    "dataset/programs_processed_precond_nograb_morepreconds/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file1003_2.txt"
)

In [30]:
description

'go to the home office and write an email'

In [21]:
preconds = add_preconds.get_preconds_script(script_str).printCondsJSON()

In [22]:
# info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=graph_input)
helper = utils.graph_dict_helper(max_nodes=check_programs.max_nodes)

In [26]:
message, executable, final_state, graph_state_list, id_mapping, info, modif_script = check_programs.check_one_program(
        helper, script, preconds, graph_input, w_graph_list=True, modify_graph=True,
        id_mapping={}, place_other_objects=True)

In [32]:
executable

True

In [5]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

In [2]:
t5_tokenizer = T5TokenizerFast.from_pretrained("t5-large")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")

In [3]:
t5_tokenizer = T5TokenizerFast.from_pretrained("t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [6]:
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [57]:
seen_envs = list(range(1,6))
heldout_envs = [6,7]

In [59]:
seen_envs

[1, 2, 3, 4, 5]

In [60]:
# Eval 1: seen environments
seen_env_fns = []
# Eval 2: held-out environments
held_out_env_fns = []
# Baseline 1: goal -> actions
# Baseline 1.5: goal -> actions (NL-ified)
# Baseline 2: goal, obs -> actions
# Baseline 2.5: goal, obs -> actions (NL-ified)
# Baseline 3: goal -> description -> actions
# Baseline 3.5: goal -> description -> actions (NL-ified)

In [61]:
for scene_dir in glob.glob("dataset/programs_processed_precond_nograb_morepreconds/executable_programs/*"):
    for file in glob.glob(os.path.join(scene_dir, "results_intentions_march-13-18", "*")):
        all_data_fns.append(file)
        scene_id = int(os.path.split(scene_dir)[-1].split('_')[0].split('TrimmedTestScene')[-1])
        if scene_id in seen_envs:
            seen_env_fns.append(file)
        else:
            held_out_env_fns.append(file)
#     title, description, script, script_str = read_script_file(
#         f"dataset/programs_processed_precond_nograb_morepreconds/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file1003_2.txt"
#     )

In [62]:
len(seen_env_fns)

2573

In [99]:
import random
random.shuffle(seen_env_fns)
train_fns = seen_env_fns[:int(.8*len(seen_env_fns))]
eval_data_seen_env = seen_env_fns[int(.8*len(seen_env_fns)):]
eval_data_new_env = held_out_env_fns
random.shuffle(eval_data_new_env)

In [100]:
print(len(train_fns), len(eval_data_seen_env), len(eval_data_new_env))

2058 515 1520


In [ ]:
class TranscriptDataEntry:
    state

In [134]:
class TranscriptStepDataEntry:
    def __init__(
        self,
        goal: str,
        description: str,
        action: ScriptLine,
        action_str: str,
        action_history: list,  # list of Action (before `action`)
        action_str_history: list,  # list of str (before `action_str`)
        curr_state: dict,  # before executing the instruction
        state_history: list,  # list of state dicts (before `curr_state`)
        #observation: dict,
        env_id: int,
        transcript_id: str,
        filename: str,
        step_idx: int,
    ):
        self.goal = goal
        self.description = description
        self.action = action
        self.action_str = action_str
        self.action_history = action_history
        self.action_str_history = action_str_history
        self.curr_state = curr_state
        #self.observation = observation
        self.state_history = state_history
        self.env_id = env_id
        self.transcript_id = transcript_id
        self.filename = filename
        self.step_idx = step_idx
    
    def __str__(self):
        return json.dumps({
            'goal': self.goal, 'desc': self.description, 'action': self.action_str,
            'action_history': self.action_history, 'action_str_history': self.action_str_history,
            'curr_state': self.curr_state, 'state_history': state_history,
            'step_idx': self.step_idx,
        }, indent=2)

In [79]:
def load_state_dict(state_list_filename):
    with open(state_list_filename) as f:
        state_list = json.load(f)
        return state_list['graph_state_list']

In [91]:
import tqdm
from torch.utils.data import DataLoader, Dataset

In [125]:
class TranscriptDataset(Dataset):
    """
    load data
    """
    def __init__(self, filenames: list, split: str):
        # TODO where did Shuang get observations? (have to run data?)
        self.split = split
        self.data = []
        for fn in tqdm.tqdm(filenames):
            title, description, script, script_str = read_script_file(fn)
            state_fn = fn.replace('executable_programs', 'state_list').replace('.txt', '.json')
            state_list = load_state_dict(state_fn)
            env_id = int(fn.split('/')[-3].split('_')[0].split('TrimmedTestScene')[-1]) - 1
            transcript_id = os.path.split(fn)[-1].split('.')[0]
            assert len(state_list) == len(script_str) + 1
            for step in range(len(script_str)):
                entry = TranscriptStepDataEntry(
                    goal=title, description=description,
                    action=script[step], action_str=script_str[step],
                    action_history=script[:step], action_str_history=script_str[:step],
                    curr_state=state_list[step], state_history=state_list[:step],
                    env_id=env_id, transcript_id=transcript_id,
                    filename='/'.join(fn.split('/')[-3:]), step_idx=step,
                )
                self.data.append(entry)

    def __getitem__(self, i):
        return self.data[i]
    
    def __len__(self):
        return len(self.data)
#     data_entry = {
#         'states': states,
#         'fn': fn,
#         'env_id': env_id,
#         'transcript_id': transcript_id,
#     }
#     data.append(data_entry)

In [126]:
train_data = TranscriptDataset(train_fns[:10], 'train')


100%|█████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.53it/s]

In [132]:
train_data[2].action_str_history

['[FIND] <spectacles> (1)', '[GRAB] <spectacles> (1)']

In [ ]:
# TODO move to a separate utilities file
def convert_action_to_nl(action: ScriptLine):
    # [FIND] <item> (1)  ->  find item 1.
    # [PUTIN] <obj> (1) <subj> (2)  ->  put obj in subj
    
    return action_str_nl

    
def convert_nl_to_action(action_nl: str):
    return action_str

In [145]:
class TranscriptDataLoader(DataLoader):
    def __init__(
        self, dataset: TranscriptDataset,
        batch_size, tokenizer, device='cpu',
        get_nlified_actions=False,
    ):
        super().__init__(dataset, batch_size, collate_fn=self.collate_fn)
        self.tokenizer = tokenizer
        self.device = device
        self.get_nlified_actions = get_nlified_actions
    
    def collate_fn(self, batch):
        batch_goal = self.tokenizer(
            [ex.goal for ex in batch], return_tensors='pt', padding=True, truncation=True,
        )
        batch_description = self.tokenizer(
            [ex.description for ex in batch], return_tensors='pt', padding=True, truncation=True,
        )
        batch_action = self.tokenizer(
            [ex.action_str for ex in batch], return_tensors='pt', padding=True, truncation=True,
        ).to(self.device)
        batch_action_history = self.tokenizer(
            ['. '.join(ex.action_str_history) for ex in batch], return_tensors='pt', padding=True, truncation=True,
        ).to(self.device)
        # TODO how to encode states??
        batch_states = [ex.curr_state for ex in batch]
#         self.tokenizer(
#             [ex.curr_state for ex in batch], return_tensors='pt', padding=True, truncation=True,
#         )
        return_batch = {
            'goals': batch_goal,
            'descriptions': batch_description,
            'actions': batch_action,
            'prev_actions': batch_action_history,
            'states': batch_states,
        }
        
        if self.get_nlified_actions:
            import pdb; pdb.set_trace()
            return_batch['actions_nl'] = self.tokenizer(
                [convert_action_to_nl(ex.action) for ex in batch], return_tensors='pt', padding=True, truncation=True,
            ).to(self.device)
            return_batch['prev_actions_nl'] = self.tokenizer([
                '. '.join([
                    convert_action_to_nl(action) for action in ex.action_history
                ]) for ex in batch], return_tensors='pt', padding=True, truncation=True,
            ).to(self.device)
        
        return return_batch
        

In [146]:
train_dl = TranscriptDataLoader(
    train_data, 5, t5_tokenizer, 'cpu', get_nlified_actions=False,
)

In [121]:
from torch.optim import AdamW

all_parameters = [p for p in t5_model.parameters() if p.requires_grad]
all_parameters = list(set(all_parameters))
optimizer = AdamW(all_parameters, lr=1e-5)

In [ ]:
def eval_model(model, eval_dl, metrics):
    for batch in tqdm(eval_dl):
        model()

In [165]:

def make_batch_inputs_tgts(
    batch_prev_actions,
    batch_curr_actions,
    batch_states,
    decoder_start_token_id: int,
    pad_token_id: int,
):    
    # TODO include environment in `decoder_input_ids` (just the initial state?)
    # [bos, batch_prev_actions['input_ids']] -> batch_curr_actions['input_ids'][0]
    # [bos, batch_prev_actions['input_ids'], batch_curr_actions['input_ids'][0]] -> batch_curr_actions['input_ids'][1]
    # [bos, batch_prev_actions['input_ids'], batch_curr_actions['input_ids'][0:1]]  -> batch_curr_actions['input_ids'][2]
    
    # 1. concatenate prev_actions and curr_actions
    # 2. 

    bs = batch_prev_actions['input_ids'].size(0)
    
    decoder_input_ids = torch.cat([
        torch.full((bs,1), decoder_start_token_id),
        batch_prev_actions['input_ids'],
    ], dim=-1)
    decoder_attention_mask = torch.cat([
        torch.ones((bs,1), dtype=batch_prev_actions['attention_mask'].dtype),
        batch_prev_actions['attention_mask'],
    ], dim=-1)
    import pdb; pdb.set_trace()
    
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[..., 1:] = input_ids[..., :-1].clone()
    shifted_input_ids[..., 0] = decoder_start_token_id
    
    labels = batch_curr_actions
    decoder_input_ids = torch.cat([
        batch_prev_actions['input_ids'],
        t5_model._shift_right(batch_curr_actions['input_ids']),
    ])
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    return decoder_input_ids, decoder_attention_mask, labels

make_batch_inputs_tgts(
    {'input_ids': torch.rand(4, 100), 'attention_mask': torch.ones(4, 100)},
    {'input_ids': torch.rand(4, 10), 'attention_mask': torch.ones(4, 10)},
    {}, 0, 1,
)

> /var/folders/c1/xkd4z0yn0fb9gy7nj5w991lh0000gn/T/ipykernel_35736/1525404404.py(15)make_batch_inputs_tgts()
     13 
     14     import pdb; pdb.set_trace()
---> 15     decoder_input_ids = torch.cat([
     16         torch.full((bs,1), decoder_start_token_id),
     17         batch_prev_actions['input_ids'],

ipdb> n
> /var/folders/c1/xkd4z0yn0fb9gy7nj5w991lh0000gn/T/ipykernel_35736/1525404404.py(16)make_batch_inputs_tgts()
     14     import pdb; pdb.set_trace()
     15     decoder_input_ids = torch.cat([
---> 16         torch.full((bs,1), decoder_start_token_id),
     17         batch_prev_actions['input_ids'],
     18     ], dim=-1)

ipdb> n
> /var/folders/c1/xkd4z0yn0fb9gy7nj5w991lh0000gn/T/ipykernel_35736/1525404404.py(17)make_batch_inputs_tgts()
     15     decoder_input_ids = torch.cat([
     16         torch.full((bs,1), decoder_start_token_id),
---> 17         batch_prev_actions['input_ids'],
     18     ], dim=-1)
     19     decoder_attention_mask = torch.cat([

ipdb> n
> /

BdbQuit: 

In [159]:
EPOCHS = 10
lang_loss = torch.tensor(float("inf"))
loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
# train model
for epoch in range(EPOCHS):
    print(f"Epoch {epoch}")
    for batch in tqdm.tqdm(train_dl, desc=str(lang_loss.item())):
        optimizer.zero_grad()
        decoder_input_ids, decoder_attention_mask, labels = make_batch_inputs_tgts(
            batch['prev_actions'],
            batch['actions'],
            batch['states'],
            decoder_start_token_id = t5_model.config.decoder_start_token_id,
            pad_token_id = t5_model.config.pad_token_id
        )
        import pdb; pdb.set_trace()
        return_dict = t5_model(
            input_ids=batch['goals']['input_ids'],
            attention_mask=batch['goals']['attention_mask'],
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
            return_dict=True,
        )
        loss = loss_fct(
            return_dict.logits.view(-1, return_dict.logits.size(-1)),
            batch['actions']['input_ids'].view(-1),
        )
        try:
            return_dict = t5_model(
                input_ids=batch['goals']['input_ids'],
                attention_mask=batch['goals']['attention_mask'],
                decoder_input_ids=batch['prev_actions']['input_ids'],
                decoder_attention_mask=batch['prev_actions']['attention_mask'],
                labels=batch['actions']['input_ids'],
                return_dict=True,
            )
        except:
            import pdb; pdb.set_trace()
            return_dict = t5_model(input_ids=batch['goals']['input_ids'], attention_mask=batch['goals']['attention_mask'], decoder_input_ids=batch['prev_actions']['input_ids'], decoder_attention_mask=batch['prev_actions']['attention_mask'], labels=batch['actions']['input_ids'], return_dict=True,)
        lang_loss, dec_output, encoder_hidden = return_dict.loss, return_dict.logits, return_dict.encoder_last_hidden_state
        lang_loss = lang_loss.mean()
        lang_loss.backward()
        optimizer.step()











inf:   0%|                                                                                             | 0/14 [00:00<?, ?it/s]

Epoch 0
> /var/folders/c1/xkd4z0yn0fb9gy7nj5w991lh0000gn/T/ipykernel_35736/3169091637.py(13)make_batch_inputs_tgts()
     11     # [bos, batch['prev_actions']['input_ids'], batch['actions']['input_ids'][0:1]]  -> batch['actions']['input_ids'][2]
     12     import pdb; pdb.set_trace()
---> 13     shifted_input_ids = input_ids.new_zeros(input_ids.shape)
     14     shifted_input_ids[..., 1:] = input_ids[..., :-1].clone()
     15     shifted_input_ids[..., 0] = decoder_start_token_id

ipdb> decoder_start_token_id
0
ipdb> pad_token_id
0
ipdb> q


BdbQuit: 